<a href="https://colab.research.google.com/github/HansHenseler/masdav2023/blob/main/Part_4_Elasticsearch_and_log2timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opensearch and log2timeline

Exercise 4:

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2023


## 1 Installing plaso tools in the colab notebook

First install Plaso-tools as we did in exercise 3

In [ ]:
# various install steps to install plaso tools and dependencies to get plaso working in colab
# -y option is to skip user interaction
# some packages need to be deinstalled and reinstalled to resolve dependencies
# these steps take app. 3 minutes to complete on a fresh colab instance
!add-apt-repository -y ppa:gift/stable
!apt update
!apt-get update
!apt install plaso-tools
!pip uninstall -y pytsk3
!pip install pytsk3
!pip uninstall -y yara-python
!pip install yara-python
!pip uninstall -y lz4
!pip install lz4
!pip install pyparsing==3.1.0

Repository: 'deb https://ppa.launchpadcontent.net/gift/stable/ubuntu/ jammy main'
Description:
Periodic releases, contains periodic releases intended for non-development use
More info: https://launchpad.net/~gift/+archive/ubuntu/stable
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/gift-ubuntu-stable-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/gift-ubuntu-stable-jammy.list
Adding key to /etc/apt/trusted.gpg.d/gift-ubuntu-stable.gpg with fingerprint 3ED1EAECE81894B171D7DA5B5E80511B10C598B8
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_

In [ ]:
# This notebook was tested with version 20220724 (psort.py assumes opensearch and no longer supports elasticsearch)

!psort.py -V

plaso - psort version 20230717


In [ ]:
# check if plaso tools were installed by running psort.py

!psort.py -o list



******************************** Output Modules ********************************
         Name : Description
--------------------------------------------------------------------------------
      dynamic : Dynamic selection of fields for a separated value output
                format.
         json : Saves the events into a JSON format.
    json_line : Saves the events into a JSON line format.
          kml : Saves events with geography data into a KML format.
       l2tcsv : CSV format used by legacy log2timeline, with 17 fixed fields.
       l2ttln : Extended TLN 7 field | delimited output.
         null : Output module that does not output anything.
   opensearch : Saves the events into an OpenSearch database.
opensearch_ts : Saves the events into an OpenSearch database for use with
                Timesketch.
        rawpy : native (or "raw") Python output.
          tln : TLN 5 field | delimited output.
         xlsx : Excel Spreadsheet (XLSX) output
----------------------------

## 2 Download and setup the Opensearch instance

There are different was to install and use Opensearch. Because we are working in a notebook we will download and install using a tar-ball which can be downloaded from: https://opensearch.org/downloads.html .

In [ ]:
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.2.0/opensearch-2.2.0-linux-x64.tar.gz
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.2.0/opensearch-2.2.0-linux-x64.tar.gz.sha512
!tar -zxf opensearch-2.2.0-linux-x64.tar.gz
!shasum -a 512 -c opensearch-2.2.0-linux-x64.tar.gz.sha512

opensearch-2.2.0-linux-x64.tar.gz: OK


In [ ]:
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.9.0/opensearch-2.9.0-linux-x64.tar.gz
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.9.0/opensearch-2.9.0-linux-x64.tar.gz.sha512
!tar -zxf opensearch-2.9.0-linux-x64.tar.gz
!shasum -a 512 -c opensearch-2.9.0-linux-x64.tar.gz.sha512

opensearch-2.9.0-linux-x64.tar.gz: OK


In [ ]:
!whoami

root


In [ ]:
# change the owner of the Opensearch filetree from root to daemon because Opensearch cannot run as root.

!sudo chown -R daemon:daemon opensearch-2.9.0/

Run Elasticsearch as a daemon process

In [ ]:
# start Opensearch from the commandline as user daemon.

!sudo -H -u daemon sh -c  "opensearch-2.9.0/opensearch-tar-install.sh 1> /content/opensearch-2.9.0/logs/os.log 2> /content/opensearch-2.9.0/logs/os.err &"

In [ ]:
# Sleep for few seconds to let the instance start.
import time
time.sleep(20)

Once the instance has been started, grep for elasticsearch in the processes list to confirm the availability.

In [ ]:
!ps -ef | grep opensearch

daemon     10959       1 99 20:41 ?        00:00:41 /content/opensearch-2.9.0/jdk/bin/java -Xshare:auto -Dopensearch.networkaddress.cache.ttl=60 -Dopensearch.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/opensearch-4849347765434272014 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Dclk.tck=100 -Djdk.attach.allowAttachSelf=true -Djava.security.policy=/content

Query the base endpoint at localhost port 9200 to retrieve information about the cluster. Opensearch requires https, username and password.

In [ ]:
!curl -XGET https://localhost:9200 -u 'admin:admin' --insecure

{
  "name" : "7cc423b2e88a",
  "cluster_name" : "opensearch",
  "cluster_uuid" : "0M_fUWGPRYygfiKZ_kVTXQ",
  "version" : {
    "distribution" : "opensearch",
    "number" : "2.9.0",
    "build_type" : "tar",
    "build_hash" : "1164221ee2b8ba3560f0ff492309867beea28433",
    "build_date" : "2023-07-18T21:23:29.367080729Z",
    "build_snapshot" : false,
    "lucene_version" : "9.7.0",
    "minimum_wire_compatibility_version" : "7.10.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "The OpenSearch Project: https://opensearch.org/"
}


In [ ]:
# After installation there is only one index that is the  .opendistro_security index that is used for internal purposes

!curl -XGET "https://localhost:9200/_cat/indices?v" -u 'admin:admin' --insecure

health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opensearch-observability VP-j2y3vQb2PMRfkqHzjgg   1   0          0            0       208b           208b
yellow open   .plugins-ml-config        68m1_jReRhGYPxMu9_pIyA   1   1          1            0      3.8kb          3.8kb
green  open   .opendistro_security      cTtG-gIaSIWIx2vG_P728A   1   0         10            0     74.7kb         74.7kb


## 3 Use Log2timeline.py and Psort.py to load data in Elasticsearch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# In part 3 (step 3) we stored the mus2019ctf.plaso file in your drive.
#
plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#
# and check if it's there
#
!ls -l $plaso_file

-rw------- 1 root root 494424064 Aug 10 21:04 gdrive/MyDrive/mus2019ctf.plaso


In [ ]:
# If it's not there you can create it by repeating the following steps
#
# The complete mus2019ctf.plaso file is 450MB and takes a while. After you have created it
# it makes sense to store it in your gdrive so you can reuse it:
#
plaso_file = 'gdrive/MyDrive/Colab\ Notebooks/Data\ Analytics\ and\ Visualisation\ Course/mus2019ctf.plaso'
#
# if not you need to create it with log2timeline.py using the complete windows_filter.txt filter
#
# add a shortcut in your Google drive to this shared drive https://drive.google.com/drive/folders/1KUlZUl4Sy2JzgbuRW-oHjIGFClY2bl75?usp=sharing
# then mount you google drive in this colab (you need to authorize this colab to access your google drive)
#
#!pip install pyparsing==3.1.0
#disk_image = "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01"
#plaso_gdrive_folder = 'gdrive/MyDrive'
#!wget "https://raw.githubusercontent.com/mark-hallman/plaso_filters/master/filter_windows.txt"
#!log2timeline.py -f filter_windows.txt --storage-file mus2019ctf.plaso $disk_image
#!ls -l $disk_image
#!cp mus2019ctf.plaso $plaso_gdrive_folder
#plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#!ls -l $plaso_file

Use psort to write events to Elasticsearch that we setup earlier. We can use the elastic output format

In [ ]:
# Before we do that, let's take a look at the opensearch.mappings file that comes with plaso
# actually there is more in that folder that you may be interested in
#
!ls /usr/share/plaso

filter_no_winsxs.yaml  opensearch.mappings  tag_linux.txt    winevt-rc.db
filter_windows.txt     plaso-data.README    tag_macos.txt
filter_windows.yaml    presets.yaml	    tag_windows.txt
formatters	       signatures.conf	    timeliner.yaml


In [ ]:
# let's take a look at the opensearch.mappings
#
!cat /usr/share/plaso/opensearch.mappings

{
  "properties": {
    "application": {
      "type": "text",
      "fields": {
        "keyword": {"type": "keyword"}}
    },
    "data": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "doc_type": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "event_type": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "exit_status": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "facility": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "file_reference": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "file_size": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "flags": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
    "identifier": {
      "type": "text",
      "fields": {"keyword": {"type": "keyword"}}
    },
 

In [ ]:
# Field file_size has type text.
#
#    "file_size": {
#      "type": "text",
#      "fields": {"keyword": {"type": "keyword"}}
#    },
#
# Let's change the mapping for file_size to type long
#
#
#    "file_size": {
#      "type": "long"
#    },

In [ ]:
!pwd

/content


In [ ]:
# run psort.py. It takes about 13 minutes to export all rows from the 385MB plaso file to Opensearch

#
!psort.py -o opensearch --server localhost --port 9200 --opensearch-user admin --opensearch-password admin --opensearch_mappings /usr/share/plaso/opensearch.mappings --use_ssl --ca_certificates_file_path /content/opensearch-2.9.0/config/root-ca.pem --index_name newmus2019ctf $plaso_file --status_view none

2023-08-10 21:27:27,408 [INFO] (MainProcess) PID:22738 <data_location> Determined data location: /usr/share/plaso
2023-08-10 21:27:27,408 [WARNING] (MainProcess) PID:22738 <opensearch_output> Note that specifying your OpenSearch password via --opensearch_password or the environment PLASO_OPENSEARCH_PASSWORD can expose the password to other users on the system.
Processing completed.


In [ ]:
# Let's take a look again at the indices in our Elasticsearch instance
#
#!curl -X GET "https://localhost:9200/_cat/indices?format=json&pretty" -u admin:admin --insecure
!curl -X GET "https://localhost:9200/_cat/indices?v" -u admin:admin --insecure

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opensearch-observability    VP-j2y3vQb2PMRfkqHzjgg   1   0          0            0       208b           208b
yellow open   .plugins-ml-config           68m1_jReRhGYPxMu9_pIyA   1   1          1            0      3.9kb          3.9kb
yellow open   security-auditlog-2023.08.10 mmqk4wJMRCe96ZSj8nf2ew   1   1         41            0    137.7kb        137.7kb
yellow open   newmus2019ctf                jCsIDdWcSRC6kLcXHUAy2w   1   1     805694            0    388.9mb        388.9mb
green  open   .opendistro_security         cTtG-gIaSIWIx2vG_P728A   1   0         10            0     75.5kb         75.5kb


In [ ]:
# we can also see what fields were mapped in this index by Psort.py
#
!curl -XGET "https://localhost:9200/newmus2019ctf/_mapping?format=json&pretty" -u admin:admin --insecure


{
  "newmus2019ctf" : {
    "mappings" : {
      "properties" : {
        "access_count" : {
          "type" : "long"
        },
        "account_rid" : {
          "type" : "long"
        },
        "application" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "birth_droid_file_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "birth_droid_volume_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "build_number" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }

## 4 Accessing Elasticsearch via the REST API

In [ ]:
# we can use the elastic search api to get the first 10 results
#
!curl -sX GET "https://localhost:9200/_search?format=json&pretty" -u admin:admin --insecure

{
  "took" : 32,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 10000,
      "relation" : "gte"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "newmus2019ctf",
        "_id" : "iHJZ4YkB_mxBwyjsJvzE",
        "_score" : 1.0,
        "_source" : {
          "data_type" : "windows:registry:key_value",
          "key_path" : "HKEY_LOCAL_MACHINE\\System\\DriverDatabase\\DriverPackages\\usbcir.inf_amd64_387d117b529b493f\\Descriptors\\USB\\VID_413C&PID_8123",
          "values" : [
            [
              "Configuration",
              "REG_SZ",
              "usbcir_Device_V2.NT"
            ],
            [
              "Manufacturer",
              "REG_SZ",
              "%microsoft%"
            ],
            [
              "Description",
              "REG_SZ",
              "%usbcir.devicedesc%"
            ]
          ],
          "path_s

In [ ]:
# show the settings for the newmus2019ctf index

!curl -sX GET "https://localhost:9200/newmus2019ctf/_settings?format=json&pretty" -u admin:admin --insecure

{
  "newmus2019ctf" : {
    "settings" : {
      "index" : {
        "replication" : {
          "type" : "DOCUMENT"
        },
        "number_of_shards" : "1",
        "provided_name" : "newmus2019ctf",
        "creation_date" : "1691702848030",
        "number_of_replicas" : "1",
        "uuid" : "jCsIDdWcSRC6kLcXHUAy2w",
        "version" : {
          "created" : "136307827"
        }
      }
    }
  }
}


## 5 Accessing the Opensearch API in Python

In [ ]:
# So far we have been accessing information directly with curl from the Opensearch REST API
# The is also an Opensearch Python API that we can use. See https://opensearch.org/docs/latest/clients/python/
#

from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
ca_certs_path = '/content/opensearch-2.9.0/config/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)


Which indexes are available?

In [ ]:
client.indices.get_alias("*")

{'.opensearch-observability': {'aliases': {}},
 '.plugins-ml-config': {'aliases': {}},
 '.opensearch-sap-pre-packaged-rules-config': {'aliases': {}},
 '.opensearch-sap-log-types-config': {'aliases': {}},
 'security-auditlog-2023.08.10': {'aliases': {}},
 'newmus2019ctf': {'aliases': {}},
 '.opendistro_security': {'aliases': {}}}

Search the index with a full-text query and get the first 5 results

In [ ]:

response = client.search(index="newmus2019ctf", body={"query": {"match": {"message": { "query": "selmabouvier"  }}}}, size=5)
docs = response['hits']['hits']
docs

[{'_index': 'newmus2019ctf',
  '_id': 'zXhc4YkB_mxBwyjsJUp_',
  '_score': 7.8139353,
  '_source': {'data_type': 'fs:stat',
   'display_name': 'NTFS:\\Users\\SelmaBouvier\\Desktop\\desktop.ini',
   'file_entry_type': 'file',
   'file_size': 282,
   'file_system_type': 'NTFS',
   'filename': '\\Users\\SelmaBouvier\\Desktop\\desktop.ini',
   'inode': '1125899906949326',
   'is_allocated': True,
   'path_spec': '{"__type__": "PathSpec", "mft_attribute": 1, "mft_entry": 106702, "location": "\\\\Users\\\\SelmaBouvier\\\\Desktop\\\\desktop.ini", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator": "TSK_PARTITION"}, "type_indicator": "NTFS"}',
   'sha256_hash': '4b9d687ac625690fd026ed4b236dad1cac90ef69e7ad256cc42766a065b50026',
   'd

In [ ]:
# The '_source' property has the psort values
#
for num, doc in enumerate(docs):
  print(num, '-->', doc['_source'], "\n")

0 --> {'data_type': 'fs:stat', 'display_name': 'NTFS:\\Users\\SelmaBouvier\\Desktop\\desktop.ini', 'file_entry_type': 'file', 'file_size': 282, 'file_system_type': 'NTFS', 'filename': '\\Users\\SelmaBouvier\\Desktop\\desktop.ini', 'inode': '1125899906949326', 'is_allocated': True, 'path_spec': '{"__type__": "PathSpec", "mft_attribute": 1, "mft_entry": 106702, "location": "\\\\Users\\\\SelmaBouvier\\\\Desktop\\\\desktop.ini", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator": "TSK_PARTITION"}, "type_indicator": "NTFS"}', 'sha256_hash': '4b9d687ac625690fd026ed4b236dad1cac90ef69e7ad256cc42766a065b50026', 'datetime': '2019-02-25T20:47:01.374840+00:00', 'message': 'NTFS:\\Users\\SelmaBouvier\\Desktop\\desktop.ini Type: file', 's

In [ ]:
# We define a Python function to list results
#
def print_results(response):
  for num, doc in enumerate(response['hits']['hits']):
    print(num, '-->', doc['_source'])

def print_results_detailed(response):
  for num, doc in enumerate(response['hits']['hits']):
    print('\n---------------------------------------------------------------------------------------------------\nresult numer: ',num)
    for key, val in doc['_source'].items():
      print(key, val)

In [ ]:
# we can try this function on the response we got earlier
#
print_results_detailed(response)


---------------------------------------------------------------------------------------------------
result numer:  0
data_type fs:stat
display_name NTFS:\Users\SelmaBouvier\Desktop\desktop.ini
file_entry_type file
file_size 282
file_system_type NTFS
filename \Users\SelmaBouvier\Desktop\desktop.ini
inode 1125899906949326
is_allocated True
path_spec {"__type__": "PathSpec", "mft_attribute": 1, "mft_entry": 106702, "location": "\\Users\\SelmaBouvier\\Desktop\\desktop.ini", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator": "TSK_PARTITION"}, "type_indicator": "NTFS"}
sha256_hash 4b9d687ac625690fd026ed4b236dad1cac90ef69e7ad256cc42766a065b50026
datetime 2019-02-25T20:47:01.374840+00:00
message NTFS:\Users\SelmaBouvier\Desktop\de

In [ ]:
# Opensearch query syntax is quite elaborate. We will provide some examples in this colab
# notice that results can have different fields. It depends on the data_type
# For a complete overview see the Opensearch reference documents
#
# https://opensearch.org/docs/latest/opensearch/query-dsl/index/
#
#query = '{"query": { "query_string": {"query": "source_short: WEBHIST"  }}}'
query = '{"query": { "query_string": {"query": "data_type: windows*link"  }}}'
#query = '{"query": { "query_string": {"query": "drive_type: 3"  }}}'
#query = '{"query": { "query_string": {"query": "drive_type: 3 AND data_type: windows*link" }}}'
#query = '{"query": { "range": {"drive_type": { "gte":0 , "lte":2 } }}}'
#query = '{"query": { "range": {"drive_type": { "gte":1 , "lte":3 } }}}'
#query = '{"query": { "query_string": {"query": "drive_type:>=0 and drive_type:<2"  }}}'
#query = '{"query": { "query_string": {"query": "file_size:>100000 "  }}}'
#query = '{"query": { "query_string": {"query": "file_size:<29696"  }}}'
#query = '{"query": { "range": {"file_size": { "gte":10000, "lte":100000 }  }}}'
#query = '{"query": { "query_string": {"query": "data_type: msie\\\\:*"  }}}'


response = client.search(index="newmus2019ctf", body=query, size=15)

print_results_detailed(response)



---------------------------------------------------------------------------------------------------
result numer:  0
data_type windows:lnk:link
drive_serial_number 3438183451
drive_type 3
file_attribute_flags 0
file_size 0
link_target <My Computer> {b4bfcc3a-db2c-424c-b029-7fe99a87c641}\EvenMoreSecretStuff.vhd
local_path C:\Users\SelmaBouvier\Desktop\EvenMoreSecretStuff.vhd
relative_path ..\..\..\..\..\Desktop\EvenMoreSecretStuff.vhd
volume_label 
working_directory C:\Users\SelmaBouvier\Desktop
path_spec {"__type__": "PathSpec", "mft_attribute": 2, "mft_entry": 97897, "location": "\\Users\\SelmaBouvier\\AppData\\Roaming\\Microsoft\\Windows\\Recent\\EvenMoreSecretStuff.lnk", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator"

## 6 Elasticsearch field aggregation

In [ ]:
# First we define some helper functions:

def print_facets(agg_dict):
  sum=0
  for field, val in agg_dict:
      print("facets of field ", field,':')
      for bucket in val['buckets']:
        for key in bucket:
          if key=='key':
            print('\t',bucket[key],end='=')
          else:
            print(bucket[key],end='')
            sum = sum + bucket[key]
        print()
      print("total number of hits for ",field," is ",sum)

def print_hit_stats(response):
  print('hit stats:')
  for key, val in response['hits'].items():
      print(key, val)
  print('\n')


In [ ]:
querystring = '{ "query_string": {"query": "source_short: WEBHIST"  }}'

query = '{"query": %s}' % querystring

print(query)


{"query": { "query_string": {"query": "source_short: WEBHIST"  }}}


In [ ]:
# Aggregating results is one of the most powerful options in Elasticsearch
#
# https://opensearch.org/docs/latest/opensearch/aggregations/
#
querystring = '{ "query_string": {"query": "SelmaBouvier"  }}'
facets = '"aggs": { "data_type": { "terms": { "field": "data_type.keyword"}}}'
query = '{"query": %s,%s}' % (querystring,facets)

print(query)


{"query": { "query_string": {"query": "SelmaBouvier"  }},"aggs": { "data_type": { "terms": { "field": "data_type.keyword"}}}}


In [ ]:
response = client.search(index="newmus2019ctf", body=query, size=0)
print_hit_stats(response)

print_facets(response['aggregations'].items())
# print_results(response)

hit stats:
total {'value': 3880, 'relation': 'eq'}
max_score None
hits []


facets of field  data_type :
	 windows:evtx:record=2698
	 msie:webcache:container=523
	 fs:stat=220
	 windows:prefetch:execution=172
	 windows:lnk:link=53
	 windows:registry:key_value=44
	 msie:webcache:cookie=40
	 msie:webcache:containers=31
	 windows:registry:appcompatcache=27
	 windows:distributed_link_tracking:creation=25
total number of hits for  data_type  is  3833


In [ ]:
# Aggregate accross multiple facets
#
querystring = '{ "query_string": {"query": "SelmaBouvier"  }}'

facets = '"aggs": { "parser": { "terms": { "field": "parser.keyword"}},  "data_type": { "terms": { "field": "data_type.keyword"}}}'
query = '{"query": %s,%s}' % (querystring,facets)

response = client.search(index="newmus2019ctf", body=query, size=0)

print_facets(response['aggregations'].items())


facets of field  parser :
total number of hits for  parser  is  0
facets of field  data_type :
	 windows:evtx:record=2698
	 msie:webcache:container=523
	 fs:stat=220
	 windows:prefetch:execution=172
	 windows:lnk:link=53
	 windows:registry:key_value=44
	 msie:webcache:cookie=40
	 msie:webcache:containers=31
	 windows:registry:appcompatcache=27
	 windows:distributed_link_tracking:creation=25
total number of hits for  data_type  is  3833


In [ ]:
# aggregate accross datetime
#
# also see pipeline aggregations https://opensearch.org/docs/latest/opensearch/pipeline-agg/

querystring = '{ "query_string": {"query": "\\\<\\\/Event\\\>"  }}'

facets = '"aggs": { "datetime": { "date_histogram": { "field": "datetime", "calendar_interval": "year"}}}'

query = '{"query": %s,%s}' % (querystring,facets)

response = client.search(index="newmus2019ctf", body=query, size=0)

response

{'took': 417,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'datetime': {'buckets': [{'key_as_string': '1970-01-01T00:00:00.000Z',
     'key': 0,
     'doc_count': 2},
    {'key_as_string': '1971-01-01T00:00:00.000Z',
     'key': 31536000000,
     'doc_count': 0},
    {'key_as_string': '1972-01-01T00:00:00.000Z',
     'key': 63072000000,
     'doc_count': 0},
    {'key_as_string': '1973-01-01T00:00:00.000Z',
     'key': 94694400000,
     'doc_count': 0},
    {'key_as_string': '1974-01-01T00:00:00.000Z',
     'key': 126230400000,
     'doc_count': 0},
    {'key_as_string': '1975-01-01T00:00:00.000Z',
     'key': 157766400000,
     'doc_count': 0},
    {'key_as_string': '1976-01-01T00:00:00.000Z',
     'key': 189302400000,
     'doc_count': 0},
    {'key_as_string': '1977-01-01T00:00:00.000Z',
     'key': 220924800000,
     'doc_count'

In [ ]:
# aggregate accross file_size (must have changed type text to long in mappings file before psort.py)
#
# also see pipeline aggregations https://opensearch.org/docs/latest/opensearch/pipeline-agg/

querystring = '{ "query_string": {"query": "file_size:<100000000"  }}'

facets = '"aggs": { "file_size": { "histogram": { "field": "file_size", "interval": 10000000}}}'

query = '{"query": %s,%s}' % (querystring,facets)

response = client.search(index="newmus2019ctf", body=query, size=0)

print_facets(response['aggregations'].items())

RequestError: ignored

In [ ]:
# date range search

query = '{"query": { "query_string": {"query": "datetime:[2019-03-12 TO 2019-03-22]"  }}}'

print(query)
response = client.search(index="newmus2019ctf", body=query, size=10)
print_results_detailed(response)


{"query": { "query_string": {"query": "datetime:[2019-03-12 TO 2019-03-22]"  }}}

---------------------------------------------------------------------------------------------------
result numer:  0
data_type fs:ntfs:usn_change
file_attribute_flags 32
filename PhotosVideoEditorSplashScreen.scale-100.png
file_reference 562949953540191
parent_file_reference 1688849860355705
offset 2184
update_reason_flags 2147549184
update_sequence_number 531543976
update_source_flags 0
path_spec {"__type__": "PathSpec", "mft_attribute": 1, "mft_entry": 83366, "location": "\\$Extend\\$UsnJrnl", "data_stream": "$J", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator": "TSK_PARTITION"}, "type_indicator": "NTFS"}
datetime 2019-03-14T09:10:11.26513

## 7 Putting Elasticsearch json out into a Pandas dataframe

In [ ]:
# the out is json format which we can store in a pandas dataframe
import pandas as pd
import json
from io import StringIO

output = !curl -sX GET "https://localhost:9200/_search?q=logon" -u admin:admin --insecure
df = pd.read_json(StringIO(output[0]))


In [ ]:
df.head

<bound method NDFrame.head of             took  timed_out  _shards  \
total        177      False      5.0   
successful   177      False      5.0   
skipped      177      False      0.0   
failed       177      False      0.0   
max_score    177      False      NaN   
hits         177      False      NaN   

                                                         hits  
total                        {'value': 431, 'relation': 'eq'}  
successful                                                NaN  
skipped                                                   NaN  
failed                                                    NaN  
max_score                                           17.892853  
hits        [{'_index': 'newmus2019ctf', '_id': 'AnZa4YkB_...  >

In [ ]:
df['hits']['hits'][:1]

[{'_index': 'newmus2019ctf',
  '_id': 'AnZa4YkB_mxBwyjs6U2_',
  '_score': 17.892853,
  '_source': {'data_type': 'windows:registry:key_value',
   'key_path': 'HKEY_LOCAL_MACHINE\\System\\ControlSet001\\Control\\Terminal Server\\Utilities\\change',
   'values': [['logon', 'REG_MULTI_SZ', '[0, 1, LOGON, chglogon.exe]'],
    ['port', 'REG_MULTI_SZ', '[0, 1, PORT, chgport.exe]'],
    ['user', 'REG_MULTI_SZ', '[0, 1, USER, chgusr.exe]'],
    ['winsta', 'REG_MULTI_SZ', '[1, WINSTA, chglogon.exe]']],
   'path_spec': '{"__type__": "PathSpec", "mft_attribute": 1, "mft_entry": 83339, "location": "\\\\Windows\\\\System32\\\\config\\\\SYSTEM", "parent": {"__type__": "PathSpec", "start_offset": 576716800, "location": "/p1", "part_index": 2, "parent": {"__type__": "PathSpec", "parent": {"__type__": "PathSpec", "location": "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01", "type_indicator": "OS"}, "type_indicator": "EWF"}, "type_indicator": "TSK_PARTITION"}, "type_indicator": "NTFS"}

# Exercises

## 1 Use elasticsearch to filter events in between 2019-03-12 and 2019-03-22

In [ ]:
# Your answer

## 2 Write a query that performs an aggregation on source_long and source_short (can you find the right field names?)

In [ ]:
# Your answer

## 3 Combine your date range filter from exercise 1 with facet aggregation in exercise 2

In [ ]:
# Your answer

## 4 ***Advanced*** Use opensearch facet aggregation to create a treemap visualisation of a filtered set of events in the index.

####Step 1
The source_short and source_long look interesting for visualisation let's focus on REG, LOG and FILE and run a query. Try the following to see what this looks like:

In [ ]:
facets = '"aggs": {   "source_long": { "terms": { "field": "source_long.keyword"}}, "source_short": { "terms": { "field": "source_short.keyword"}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s AND  source_short:(FILE OR LOG OR REG)"  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)
response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
print_facets(response['aggregations'].items())

Why is this not very helpful for a treemap visualisation?

In [ ]:
# Your answer

####Step 2
One approach is to aggregate only accross source_long an run 3 separate queries for source_short equal to REG, LOG and FILE respectively and then combine the results into a single dataframe that we can visualise:

In [ ]:
# Your answer. If you are struggling you can skip this step and move on to step 3.

####Step 3
Opensearch can also aggregate accros multiple fields combined. This is called multi_terms aggregation which aggregates accross all (source_long, source_short) value pairs. Note: by default opensearch returns 10 buckets max. We set it to 20 here using the size parameter.

See https://opensearch.org/docs/2.0/opensearch/bucket-agg/ for more information about opensearch and multi_term aggregation.

In [ ]:
# Your answer

####Step 4
The exercise asked for aggregation accross 3 fields. So let's add the parser field and deepen our treemap visualisation

In [ ]:
# Your answer